<a href="https://colab.research.google.com/github/sergio2pi/CEIA-FIUBA_LLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext
%pip install numpy pandas sentencepiece
%pip install transformers==4.16.2
%pip install pytorch-lightning==1.5.10

import pandas as pd
c
from sklearn.model_selection import train_test_split

Descarga del modelo

In [1]:
!git clone https://github.com/sergio2pi/CEIA-FIUBA_LLM.git
!git -C CEIA-FIUBA_LLM pull
!cp -r CEIA-FIUBA_LLM/t5 t5

Cloning into 'CEIA-FIUBA_LLM'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 0), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 4.86 MiB | 16.04 MiB/s, done.
Already up to date.


Carga del dataset

In [25]:
data_path = "CEIA-FIUBA_LLM/dataset/medquad.csv"
data = pd.read_csv(data_path)

data.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16412 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16412 non-null  object
 1   answer      16407 non-null  object
 2   source      16412 non-null  object
 3   focus_area  16398 non-null  object
dtypes: object(4)
memory usage: 513.0+ KB


In [5]:
#Función para normalizar el texto
import re
def clean_text(txt):
    new_txt = txt
    new_txt.replace("\'d", " had")
    new_txt.replace("\'s", " is")
    new_txt.replace("\'m", " am")
    new_txt.replace("don't", "do not")
    new_txt = re.sub(r'\W+', ' ', txt)

El modelo recibe un dataframe con 2 columnas, el texto de origen ("source text") y el texto objetivo (target_text):

In [27]:
if 'source' in data.columns and 'focus_area' in data.columns:
  data.drop(columns=['source', 'focus_area'])
  data = data.rename(columns={"question":"source_text", "answer":"target_text"})
  data = data[['source_text', 'target_text']]

# Se elimina las filas con valores nulos
data = data.dropna()

data.head()

,source_text,target_text
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea..."
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ..."
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...


Dataset final

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16407 entries, 0 to 16411
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_text  16407 non-null  object
 1   target_text  16407 non-null  object
dtypes: object(2)
memory usage: 384.5+ KB


In [29]:
train_df, test_df = train_test_split(data, test_size=0.2)
train_df.shape, test_df.shape

((13125, 2), (3282, 2))

Entrenamiento del modelo

In [36]:
# Esto para limpiar la RAM
%pip install koila
from koila import lazy
input = lazy(input, batch=0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.9 MB/s eta 0:00:00


In [37]:
from importlib import reload
import t5
#%load_ext autoreload
#%autoreload 2

model = t5.SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=50,
            target_max_token_len=250,
            batch_size=16,
            max_epochs=3,
            use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
!cp -r outputs drive/MyDrive/LLM/

In [40]:
!cp -r lightning_logs/ drive/MyDrive/LLM/

In [41]:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-1.9027-val-loss-1.7201", use_gpu=True)

In [43]:
input_text="What can cause heart failure?"
model.predict(input_text)

['What causes heart failure? Heart failure is caused by changes (mutations) in the X-linked gene. This gene provides instructions for making a protein called x-linked polymerase chain reaction (X-linked polymerase chain reaction). The X-linked polymerase chain reaction (X-linked polymerase chain reaction) is one of the two major types of cardiomyopathy. It is involved in the production of proteins that help control the flow of blood from the heart to other parts of the body. In some cases, this protein can be converted to a form of protein that helps control the flow of blood from the heart to other parts of the body. A lack of X-linked polymerase chain reaction may lead to heart failure.']

In [44]:
input_text="Can you tell me how to treat Alzheimer's disease?"
model.predict(input_text)

["How might Alzheimer's disease be treated? There is no specific treatment for Alzheimer's disease. Treatment may include medications, such as antidepressants, a drug that helps control the risk of developing dementia, and/or other conditions."]

In [45]:
input_text="symptoms of Breast Cancer"
model.predict(input_text)

['Signs and symptoms of breast cancer include: - Alopecia - Painless lumps in the breast or on the back of the neck - Cancerous tumors - Breast pain - The most common signs and symptoms of breast cancer are fatigue, nausea, vomiting, and weight loss. Other signs and symptoms of breast cancer may be caused by breast cancer or by other conditions. Signs and symptoms of breast cancer can vary from person to person.']